In [32]:
'''
IFT6390 Fall/Automne 2018, HW1, Q4, practical
Note to TAs: this program runs on Python 3
'''

'''
Part 1
'''

import numpy as np

class gaussianDiag:
    def __init__(self, data, label):
        self.train_data = data
        self.train_label = label
        self.train_mean = 0
        self.train_cov = 0
    
    def train_diag(self):
        '''
        Generate a diagonal covariance matrix
        '''
        d = self.train_data.shape
        print(d)
        self.train_mean = np.mean(self.train_data)
        print("train_mean", self.train_mean)
        self.train_cov = np.multiply( 
            (self.train_data - self.train_mean), np.transpose((self.train_data - self.train_mean))
        )
        self.train_cov =np.diag(self.train_cov /d)
        print(self.train_cov)
        
        
        return self.train_mean, self.train_cov


    def predict_diag(self, train_label):
        '''
        calculate log density
        '''
    
        #inverse of diagonals cov
        print("np.diag(self.train_cov)", np.diag(self.train_cov))
        train_cov_inv = np.linalg.inv(self.train_cov)
        print("train_cov_inv", train_cov_inv)
        
        


data = np.array([2 ,2,3])
label = np.array([3,2,2])
part1 = gaussianDiag(data, label)

part1.train_diag()
part1.predict_diag(label)

(3,)
train_mean 2.3333333333333335
[[0.03703704 0.         0.        ]
 [0.         0.03703704 0.        ]
 [0.         0.         0.14814815]]
np.diag(self.train_cov) [0.03703704 0.03703704 0.14814815]
train_cov_inv [[27.    0.    0.  ]
 [ 0.   27.    0.  ]
 [ 0.    0.    6.75]]


In [ ]:
def train_parzen(train_data):
    pass

def predict_parzen(test_data):
    pass